# Python with PostgreSQL & PostGIS

<span>Note: Please always run the complete Jupyter Notebook from the beginning, as object names such as 'sql' and 'gdf' are reused in the code cells.</span>

## Libraries and Settings

In [6]:
# Libraries
import os
import folium
import pandas as pd
import geopandas as gpd
from sqlalchemy import create_engine, text

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

print(os.getcwd())

/workspaces/python_postgresql_postgis


## Create database connection

In [7]:
# Set up database connection
user = "pgadmin"
password = "geheim"
host = "localhost"
port = "5432"
database = "osm_switzerland"

# Create Connection URL
db_connection_url = "postgresql://" + user + ":" + password +\
                    "@" + host + ":" + port + "/" + database

# Create SQLAlchemy Engine
engine = create_engine(db_connection_url)

# Test database connection
try:
    with engine.connect() as connection:
        result = connection.execute(text("SELECT current_database();"))
        db_name = result.scalar()
        print("Current database:", db_name)
except Exception as e:
    print("Error connecting to the database:", e)

# Dispose the engine
engine.dispose()

Current database: osm_switzerland


## List tables in database

In [8]:
# Create SQLAlchemy Engine
engine = create_engine(db_connection_url)

# Open a connection
with engine.connect() as connection:

    # Execute the query
    result = connection.execute(text("""SELECT table_name
                                        FROM information_schema.tables
                                        WHERE table_schema = 'public';"""))
    
    # Fetch and print the results
    for row in result:
        print(row[0])

# Dispose the engine
engine.dispose()

geography_columns
geometry_columns
spatial_ref_sys
planet_osm_roads
planet_osm_point
planet_osm_polygon
planet_osm_line


## Show columns and data types of a selected table

In [9]:
# Create SQLAlchemy Engine
engine = create_engine(db_connection_url)

# Specify your table name
table_name = 'planet_osm_polygon'

# Query to get column information
query = f"""SELECT column_name, data_type 
        FROM information_schema.columns 
        WHERE table_name = '{table_name}'"""

# Execute the query and read the result into a DataFrame
df = pd.read_sql(query, engine)

# Dispose the engine
engine.dispose()

# Print the DataFrame
df

,column_name,data_type
0,osm_id,bigint
1,access,text
2,addr:housename,text
3,addr:street,text
4,addr:housenumber,text
...,...,...
68,width,text
69,wood,text
70,z_order,integer
71,way_area,real


## Query: Select buildings for which the full address is available in defined zip code areas

In [ ]:
# Create SQLAlchemy Engine
engine = create_engine(db_connection_url)

# Define SQL query 
sql = """SELECT
                p.osm_id,
                p."addr:street",
                p."addr:housenumber",
                p."addr:city",
                p."addr:postcode",
                p.building,
                st_transform(p.way, 4326) AS geom
        FROM
                public.planet_osm_polygon AS p
        WHERE
                p."addr:street" IS NOT NULL
                AND p."addr:housenumber" IS NOT NULL
                AND p."addr:city" IS NOT NULL
                AND p."addr:postcode" IN ('8001', '8002')"""

# Query the database and store the result in a GeoDataFrame
gdf_buildings = gpd.GeoDataFrame.from_postgis(sql, engine)

# Dispose the engine
engine.dispose()

# Print the GeoDataFrame
gdf_buildings


## Show selected features on map

<span">Note the popup field in the map, which has been added to provide additional information about buildings.</span>

<span">Example of alternative background maps (maptiles) are:</span>
- <span">EsriWorldImagery</span>
- <span">EsriWorldTopoMap</span>
- <span">EsriWorldGrayCanvas</span>
- <span">CartoDBDarkMatter</span>
- <span">CartoDBPositron</span>


In [ ]:
# Ensure the GeoDataFrame has the correct projection (EPSG:4326)
if gdf_buildings.crs is None:
    gdf_buildings.set_crs(epsg=4326, inplace=True)
else:
    pass

# Calculate the mean longitude and latitude for the map center
centroids = gdf_buildings.geometry.centroid
lon = centroids.x.mean()
lat = centroids.y.mean()

# Initialize the map
m = folium.Map(location=[lat, lon], 
               zoom_start=15,
               tiles='EsriWorldImagery')

# Map settings
folium.GeoJson(
    gdf_buildings,
    name='geojson',
    weight=0.5,
    fill_color='greenyellow',
    fillOpacity=0.8,
    popup=folium.GeoJsonPopup(fields=['addr:street',
                                      'addr:housenumber',
                                      'addr:city',
                                      'addr:postcode',
                                      'building'])
).add_to(m)

folium.LayerControl().add_to(m)

# Save map as HTML
m.save('MAPS/map_buildings.html')


## Query: Select coffee stores

In [ ]:
# Create SQLAlchemy Engine
engine = create_engine(db_connection_url)  

# Define SQL query
sql = """SELECT
            h.osm_id,
            h.shop,
            h.name,
            ST_Transform(h.way, 4326) AS geom
        FROM planet_osm_point h
        WHERE h.shop = 'coffee';"""

# Query the database and store the result in a GeoDataFrame
gdf_coffee_stores = gpd.GeoDataFrame.from_postgis(sql, engine)

# Dispose the engine
engine.dispose()

# Print the GeoDataFrame
gdf_coffee_stores.head(10)


## Show selected features on map

In [ ]:
# Ensure the GeoDataFrame has the correct projection (EPSG:4326)
if gdf_coffee_stores.crs is None:
    gdf_coffee_stores.set_crs(epsg=4326, inplace=True)
else:
    pass

# Calculate the mean longitude and latitude for the map center
centroids = gdf_coffee_stores.geometry.centroid
lon = centroids.x.mean()
lat = centroids.y.mean()

# Initialize the map
m = folium.Map(location=[lat, lon], 
               zoom_start=11, 
               tiles='EsriWorldTopoMap')

# Map settings
folium.GeoJson(
    gdf_coffee_stores,
    name='map',
    popup=folium.GeoJsonPopup(fields=['name', 'shop'])
).add_to(m)

folium.LayerControl().add_to(m)

# Save map as HTML
m.save('MAPS/map_coffee_stores.html')


## Query: Select all supermarkets in a distance of 1000m around the central station in the city of Winterthur.

<span>Note:</span>

<span>For each supermarket, the distance to the central station in meters is calculated and stored as new column 'distance_meters'.</span>

<span>In addition, a popup field was added to the map, allowing users to view detailed information about each selected feature when they click on it.</span>

<span>The WGS84 (World Geodetic System 1984) coordinates in ST_MakePoint(LON, LAT) were derived from: https://tools.retorte.ch/map.</span>


In [ ]:
# Create SQLAlchemy Engine
engine = create_engine(db_connection_url)  

# Define SQL query
sql = """SELECT
            p.osm_id,
            p.shop,
            p.name,
            ST_Distance(
                ST_Transform(p.way, 4326)::geography,
                -- Central station coordinates
                ST_SetSRID(ST_MakePoint(8.72397, 47.50031), 4326)::geography
            ) AS distance_meters,
            ST_TRANSFORM(p.way, 4326) AS geom
        FROM
            planet_osm_point AS p
        WHERE
            p.shop = 'supermarket'
            AND ST_DWithin(
                ST_Transform(p.way, 4326)::geography,
                -- Central station coordinates
                ST_SetSRID(ST_MakePoint(8.72397, 47.50031), 4326)::geography,
                1000
            )
        ORDER BY distance_meters;"""

# Query the database and store the result in a GeoDataFrame
gdf_supermarkets = gpd.GeoDataFrame.from_postgis(sql, engine)

# Dispose the engine
engine.dispose()

# Print the GeoDataFrame
gdf_supermarkets.head(10)


## Show selected features on map

In [ ]:
# Ensure the GeoDataFrame has the correct projection (EPSG:4326)
if gdf_supermarkets.crs is None:
    gdf_supermarkets.set_crs(epsg=4326, inplace=True)
else:
    pass

# Calculate the mean longitude and latitude for the map center
centroids = gdf_supermarkets.geometry.centroid
lon = centroids.x.mean()
lat = centroids.y.mean()

# Initialize the map
m = folium.Map(location=[lat, lon], 
               zoom_start=16, 
               tiles='ESRIWorldImagery')

# Map settings
folium.GeoJson(
    gdf_supermarkets,
    name='map',
    popup=folium.GeoJsonPopup(fields=['name', 'distance_meters'])
).add_to(m)

folium.LayerControl().add_to(m)

# Save map as HTML
m.save('MAPS/map_supermarkets.html')


## Query: Select all roads classified as 'motorway' and create a 5000m buffer around these roads.

In [ ]:
# Create SQLAlchemy Engine
engine = create_engine(db_connection_url)

# Define SQL query (major roads)
sql = """-- Create buffer around major roads
        SELECT 
            1 as group_id,
            ST_TRANSFORM(ST_UNION(ST_Buffer(p.way::geometry, 2000)), 4326) AS geom
        FROM public.planet_osm_roads AS p
        WHERE
            highway = 'motorway';"""

# Query the database and store the result in a GeoDataFrame
gdf_roads = gpd.GeoDataFrame.from_postgis(sql, engine, geom_col='geom')

# Dispose the engine
engine.dispose()

# Print the GeoDataFrame
gdf_roads

## Show selected features on map

In [ ]:
# Ensure the GeoDataFrame has the correct projection (EPSG:4326)
if gdf_roads.crs is None:
    gdf_roads.set_crs(epsg=4326, inplace=True)
else:
    pass

# Calculate the mean longitude and latitude for the map center
centroids = gdf_roads.geometry.centroid
lon = centroids.x.mean()
lat = centroids.y.mean()

# Initialize the map
m = folium.Map(location=[lat, lon], 
               zoom_start=11, 
               tiles='EsriWorldTopoMap')

# Map settings
folium.GeoJson(
    gdf_roads,
    name='map'
).add_to(m)

folium.LayerControl().add_to(m)

# Save map as HTML
m.save('MAPS/map_roads.html')


### Jupyter notebook --footer info-- (please always provide this at the end of each notebook)

In [ ]:
import os
import platform
import socket
from platform import python_version
from datetime import datetime

print('-----------------------------------')
print(os.name.upper())
print(platform.system(), '|', platform.release())
print('Datetime:', datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
print('Python Version:', python_version())
print('-----------------------------------')